# Helmet Detection - Faster R-CNN ResNet50 V1 640x640:

## 1. DOWNLOAD THE PRETRAINED MODELS FROM THE TENSORFLOW MODEL ZOO

In [1]:
import os

# DATA_DIR = os.path.join(os.getcwd(), 'data')
MODELS_DIR = os.path.join('pre-trained-model')
for dir in [MODELS_DIR]:
    if not os.path.exists(dir):
        os.mkdir(dir)

In [2]:
main_directory = 'models'
subfolder_name = 'faster_rcnn_resnet50_v1_640x640' #Change

main_directory_path = os.path.join(os.getcwd(), main_directory)
subfolder_path = os.path.join(main_directory_path, subfolder_name)

if not os.path.exists(subfolder_path):
    os.makedirs(subfolder_path)

In [3]:
import tarfile
import urllib.request
import os

# Download and extract model
MODEL_DATE = '20200711' #Change
MODEL_NAME = 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8' #Change
MODEL_TAR_FILENAME = MODEL_NAME + '.tar.gz'
MODELS_DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/'
MODEL_DOWNLOAD_LINK = MODELS_DOWNLOAD_BASE + MODEL_DATE + '/' + MODEL_TAR_FILENAME
PATH_TO_MODEL_TAR = os.path.join(MODELS_DIR, MODEL_TAR_FILENAME)
PATH_TO_CKPT = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'checkpoint/'))
PATH_TO_CFG = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'pipeline.config'))
if not os.path.exists(PATH_TO_CKPT):
    print('Downloading model. This may take a while... ', end='')
    urllib.request.urlretrieve(MODEL_DOWNLOAD_LINK, PATH_TO_MODEL_TAR)
    tar_file = tarfile.open(PATH_TO_MODEL_TAR)
    tar_file.extractall(MODELS_DIR)
    tar_file.close()
    os.remove(PATH_TO_MODEL_TAR)
    print('Done')

## 2. SETUP PATHS:

In [4]:
WORKSPACE_PATH = '../training_helper_directory'
SCRIPT_PATH = WORKSPACE_PATH + '/scripts'
ANNOTATION_PATH = WORKSPACE_PATH + '/annotations'

IMAGE_PATH = '../Dataset/"PASCAL VOC Format Dataset"'

CUSTOM_MODEL_NAME = '/faster_rcnn_resnet50_v1_640x640' #Change: same as subfolder name
MODEL_PATH = './models' + CUSTOM_MODEL_NAME

PRETRAINED_PATH = './pre-trained-model/' + MODEL_NAME
CONFIG_PATH = MODEL_PATH + '/pipeline.config'

EXPORT_PATH = '../exported-models/my-model'

## 3. CREATE TF RECORDS:

In [5]:
# Create train data:
!python {SCRIPT_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}

# Create test data:
!python {SCRIPT_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: ../training_helper_directory/annotations/train.record
Successfully created the TFRecord file: ../training_helper_directory/annotations/test.record


## 4. COPY MODEL CONFIG FILE TO TRANING FOLDER:

In [6]:
SOURCE_PATH = "D:\\Helmet Detection System\\Faster RCNN ResNet50 V1 640x640\\pre-trained-model\\faster_rcnn_resnet50_v1_640x640_coco17_tpu-8\\pipeline.config" #Change
DESTINATION_PATH = "D:\\Helmet Detection System\\Faster RCNN ResNet50 V1 640x640\models\\faster_rcnn_resnet50_v1_640x640" #Change

In [7]:
!copy "{SOURCE_PATH}" "{DESTINATION_PATH}"

        1 file(s) copied.


## 5. UPDATING CONFIG FOR TRANSFER LEARNING:

In [8]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [9]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [10]:
config

{'model': faster_rcnn {
   num_classes: 90
   image_resizer {
     keep_aspect_ratio_resizer {
       min_dimension: 640
       max_dimension: 640
       pad_to_max_dimension: true
     }
   }
   feature_extractor {
     type: "faster_rcnn_resnet50_keras"
     batch_norm_trainable: true
   }
   first_stage_anchor_generator {
     grid_anchor_generator {
       height_stride: 16
       width_stride: 16
       scales: 0.25
       scales: 0.5
       scales: 1.0
       scales: 2.0
       aspect_ratios: 0.5
       aspect_ratios: 1.0
       aspect_ratios: 2.0
     }
   }
   first_stage_box_predictor_conv_hyperparams {
     op: CONV
     regularizer {
       l2_regularizer {
         weight: 0.0
       }
     }
     initializer {
       truncated_normal_initializer {
         stddev: 0.009999999776482582
       }
     }
   }
   first_stage_nms_score_threshold: 0.0
   first_stage_nms_iou_threshold: 0.699999988079071
   first_stage_max_proposals: 300
   first_stage_localization_loss_weight: 2.0

In [11]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [12]:
pipeline_config.model.faster_rcnn.num_classes = 2 #Change

pipeline_config.train_config.batch_size = 1 #Change
pipeline_config.train_config.num_steps = 1000 #Change
pipeline_config.train_config.use_bfloat16 = False

pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_PATH + '/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection" 

pipeline_config.train_input_reader.label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH+ '/train.record'] 

pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [13]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:
    f.write(config_text)

## 6. TRAIN THE MODEL:

In [14]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [15]:
import os
os.getcwd()

'd:\\Helmet Detection System\\Faster RCNN ResNet50 V1 640x640'

In [16]:
!python {SCRIPT_PATH + '/model_main_tf2.py'} --model_dir={MODEL_PATH} --pipeline_config_path={MODEL_PATH + '/pipeline.config'}

2024-03-10 16:39:22.426566: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-10 16:39:22.808670: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1656 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0310 16:39:22.868163 45148 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I0310 16:39:22.875199 45148 config_util.py:552] Maybe overwriting train_steps: None
INFO:te

In [17]:
!python {SCRIPT_PATH + '/exporter_main_v2.py'} --input_type image_tensor --pipeline_config_path {CONFIG_PATH} --trained_checkpoint_dir {MODEL_PATH + '/' } --output_directory exported-models\my-model

2024-03-10 16:40:04.772087: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-10 16:40:05.121086: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1656 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
W0310 16:40:05.258622 37972 deprecation.py:623] From c:\Users\sudee\.conda\envs\helmet\lib\site-packages\tensorflow\python\autograph\impl\api.py:458:

Inferencing My Trained Models

In [18]:
"""
Object Detection (On Image) From TF2 Saved Model
=====================================
"""

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
import cv2
import argparse
# from google.colab.patches import cv2_imshow

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# PROVIDE PATH TO IMAGE DIRECTORY
IMAGE_PATHS = 'D:/Helmet Detection System/Dataset/PASCAL VOC Format Dataset/test/BikesHelmets392_png.rf.9802b10c090e222ccda1a9c575875961.jpg' #Change


# PROVIDE PATH TO MODEL DIRECTORY
PATH_TO_MODEL_DIR = './exported-models/my-model'

# PROVIDE PATH TO LABEL MAP
PATH_TO_LABELS = ANNOTATION_PATH + '/label_map.pbtxt'

# PROVIDE THE MINIMUM CONFIDENCE THRESHOLD
MIN_CONF_THRESH = float(0.60)

# LOAD THE MODEL

import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# LOAD SAVED MODEL AND BUILD DETECTION FUNCTION
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

# LOAD LABEL MAP DATA FOR PLOTTING

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.
    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.
    Args:
      path: the file path to the image
    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

print('Running inference for {}... '.format(IMAGE_PATHS), end='')

image = cv2.imread(IMAGE_PATHS)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_expanded = np.expand_dims(image_rgb, axis=0)

# The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
input_tensor = tf.convert_to_tensor(image)
# The model expects a batch of images, so add an axis with `tf.newaxis`.
input_tensor = input_tensor[tf.newaxis, ...]

# input_tensor = np.expand_dims(image_np, 0)
detections = detect_fn(input_tensor)

# All outputs are batches tensors.
# Convert to numpy arrays, and take index [0] to remove the batch dimension.
# We're only interested in the first num_detections.
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
               for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

image_with_detections = image.copy()

# SET MIN_SCORE_THRESH BASED ON YOU MINIMUM THRESHOLD FOR DETECTIONS
viz_utils.visualize_boxes_and_labels_on_image_array(
      image_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=100,
      min_score_thresh=0.5,
      agnostic_mode=False)

print('Done')
# DISPLAYS OUTPUT IMAGE
cv2.imshow('test', image_with_detections)
# CLOSES WINDOW ONCE KEY IS PRESSED
cv2.waitKey(0)
cv2.destroyAllWindows()

Loading model...

OSError: SavedModel file does not exist at: ./exported-models/my-model/saved_model\{saved_model.pbtxt|saved_model.pb}